In [ ]:
import gen3_validator
# loading gen3 bundled jsonschema into dictionary class
dd = gen3_validator.dict.DataDictionary('../examples/schema/json/schema_dev.json')
dd.parse_schema()

In [5]:
# getting list of entities
dd.get_nodes()

['demographic.yaml',
 'project.yaml',
 'acknowledgement.yaml',
 'medical_history.yaml',
 '_definitions.yaml',
 '_settings.yaml',
 'program.yaml',
 'sample.yaml',
 '_terms.yaml',
 'lipidomics_file.yaml',
 'publication.yaml',
 'subject.yaml',
 'core_metadata_collection.yaml']

In [7]:
# Returning the jsonschema for a given entity
dd.return_schema('lipidomics_file.yaml')

{'$schema': 'http://json-schema.org/draft-04/schema#',
 'id': 'lipidomics_file',
 'title': 'Lipidomics File',
 'type': 'object',
 'namespace': 'https://data.acdc.ozheart.org',
 'category': 'data_file',
 'program': '*',
 'project': '*',
 'description': 'Data file containing lipidomics data',
 'additionalProperties': False,
 'submittable': True,
 'validators': None,
 'systemProperties': ['id',
  'project_id',
  'state',
  'created_datetime',
  'updated_datetime'],
 'links': [{'exclusive': False,
   'required': True,
   'subgroup': [{'name': 'samples',
     'backref': 'lipidomics_files',
     'label': 'data_from',
     'target_type': 'sample',
     'multiplicity': 'many_to_many',
     'required': False},
    {'name': 'core_metadata_collection',
     'backref': 'lipidomics_files',
     'label': 'describes',
     'target_type': 'core_metadata_collection',
     'multiplicity': 'one_to_one',
     'required': True}]}],
 'required': ['type',
  'submitter_id',
  'core_metadata_collection',
  'da

In [1]:
!pip install dictionaryutils

  Using cached dictionaryutils-3.5.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached cdislogging-1.1.1-py3-none-any.whl
  Using cached gen3dictionary-2.0.4-py3-none-any.whl
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
Using cached dictionaryutils-3.5.1-py3-none-any.whl (16 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.25.1
    Uninstalling jsonschema-4.25.1:
      Successfully uninstalled jsonschema-4.25.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [dictionaryutils]


In [ ]:

import dictionaryutils
import json
import yaml
dd = dictionaryutils.DataDictionary('/Users/harrijh/projects/gen3schemadev/examples/schema/yaml')
dd.load_data(directory='/Users/harrijh/projects/gen3schemadev/examples/schema/yaml')
dd_resolved = dd.schema

for k, v in dd_resolved.items():
    with open(f'/Users/harrijh/projects/gen3schemadev/examples/schema/yaml/resolved/{k}.yaml', 'w') as f:
        yaml.safe_dump(v, f)


with open(f'/Users/harrijh/projects/gen3schemadev/examples/schema/json/gen3_bundled_schema_resolved.json', 'w') as f:
    json.dump(dd_resolved, f)

In [6]:
dd = dictionaryutils.load_schemas_from_file('/Users/harrijh/projects/gen3schemadev/examples/schema/json/schema_dev.json')
dd

({'demographic.yaml': {'$schema': 'http://json-schema.org/draft-04/schema#',
   'id': 'demographic',
   'title': 'Demographic',
   'type': 'object',
   'namespace': 'http://commons.heartdata.baker.edu.au/',
   'category': 'clinical',
   'program': '*',
   'project': '*',
   'description': 'Data for the characterization of the patient by means of segementing the population (e.g. characterization by age, sex, or race).',
   'additionalProperties': False,
   'submittable': True,
   'validators': None,
   'systemProperties': ['id',
    'project_id',
    'state',
    'created_datetime',
    'updated_datetime'],
   'links': [{'name': 'subjects',
     'backref': 'demographics',
     'label': 'describes',
     'target_type': 'subject',
     'multiplicity': 'one_to_one',
     'required': True}],
   'required': ['type', 'submitter_id', 'subjects'],
   'uniqueKeys': [['id'], ['project_id', 'submitter_id']],
   'properties': {'$ref': '_definitions.yaml#/ubiquitous_properties',
    'subjects': {'$r

# To validate a gen3schemadev input yaml with the input schema


In [ ]:
# !pip install check-jsonschema

In [31]:
!check-jsonschema --schemafile ../src/gen3schemadev/schema/input_schema.yml ../src/gen3schemadev/schema/input_example.yml --verbose

14655.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ok -- validation done
The following files were checked:
  ../src/gen3schemadev/schema/input_example.yml


# To validate a single gen3 schema yaml with the metaschema
- note, the schema file needs to be resolved

In [34]:
!check-jsonschema --schemafile ../src/gen3schemadev/schema/gen3_metaschema.yml ../examples/schema/yaml/resolved/lipidomics_file.yaml --verbose

14814.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ok -- validation done
The following files were checked:
  ../examples/schema/yaml/resolved/lipidomics_file.yaml


if you give a non-resolved schema, it will not validate

In [35]:
!check-jsonschema --schemafile ../src/gen3schemadev/schema/gen3_metaschema.yml ../examples/schema/yaml/lipidomics_file.yaml --verbose

14847.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Schema validation errors were encountered.
  ../examples/schema/yaml/lipidomics_file.yaml::$.properties.$ref: '_definitions.yaml#/data_file_properties' is not of type 'object'
  ../examples/schema/yaml/lipidomics_file.yaml::$.properties.$ref: '_definitions.yaml#/data_file_properties' is not of type 'object'
